This code is based on/inspired by a tutorial from Packt: https://hub.packtpub.com/use-tensorflow-and-nlp-to-detect-duplicate-quora-questions-tutorial/, written by Sunith Shetty.
The license that came with the original code can be found in the EXTERNAL_LICENSES folder.

The data in "quora_duplicate_questions.tsv" is released for non-commercial use only by Quora.
You can download this data from: http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv.
More info can be found on: https://www.quora.com/about/tos.

The pretrained Word2vec model from the Google News Corpus can be downloaded from: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz.
**Warning:** the decompressed file is appr. 3.5GB in size and will be loaded into memory so 16GB of RAM and a 64-bit Python interpreter are advised.

It is **important to note** that the data collected by Quora is a snapshot of a certain moment in history so questions about important events during that time may introduce biasses in the data. One way to obtain better models would be by collecting new domain-specific or general data, depending on the purpose of the recognizer.

It is also **important to note** that as long as the model is based of the quora data set, the model can only be used for **non-commercial** purposes. So creating a new, independent data set is of paramount importance, should you wish to use the subsequent models for commercial purposes.

quora_duplicate_questions.tsv and GoogleNews-vectors-negative300.bin.gz must be downloaded to /nlp_tools.

**This code expands the quora data set with useful features to train a machine learning model.**

In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk import word_tokenize
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy import sparse
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, \
                                   euclidean, minkowski, braycurtis
from copy import deepcopy

from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [2]:
# Download necessary nltk packages; only run this cell if you do not have
# these packages installed
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to C:\Users\Yves
[nltk_data]     D'hondt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Yves
[nltk_data]     D'hondt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**ASSUMPTIONS**
1. 2 questions that mean the same often share a lot of words, while 2 different questions rarely share a lot of words.
2. 2 questions that mean the same often have a small edit distance, while 2 different questions rarely have a small edit distance.

# Analyse & Expand the Data

In [3]:
# Read in the data and remove unnecessary columns
data = pd.read_csv("quora_duplicate_questions.tsv", sep="\t") \
         .drop(["id", "qid1", "qid2"], axis=1)
data.question1 = data.question1.apply(lambda x: str(x))
data.question2 = data.question2.apply(lambda x: str(x))

**LENGTH BASED FEATURES**

In [4]:
# Calculate the length of each sentence
data["len_q1"] = data.question1.apply(lambda x: len(x))
data["len_q2"] = data.question2.apply(lambda x: len(x))
# Calculate the difference between the lengths of each pair of questions
data["diff_len"] = data.len_q1 - data.len_q2

In [5]:
# Calculate the character length of each sentence (excluding spaces)
data["len_char_q1"] = data.question1.apply(lambda x: len(x.replace(" ", "")))
data["len_char_q2"] = data.question2.apply(lambda x: len(x.replace(" ", "")))

In [6]:
# Calculate the word count of each sentence
data["len_word_q1"] = data.question1.apply(lambda x: len(x.split()))
data["len_word_q2"] = data.question2.apply(lambda x: len(x.split()))

In [7]:
# Count the number of common words in each pair of questions
data["common_words"] = \
    data.apply(lambda x: len(set(x.question1.lower().split()).intersection(
                             set(x.question2.lower().split()))),
               axis=1)

In [8]:
# The length-based feature set for future reference
fs_1 = ['len_q1', 'len_q2', 'diff_len', 'len_char_q1',
        'len_char_q2', 'len_word_q1', 'len_word_q2',
        'common_words']

**DISTANCE BASED FEATURES**

In [9]:
# Calculate the Q and W ratio of each pair of questions
data["fuzz_QRatio"] = \
    data.apply(lambda x: fuzz.QRatio(x.question1,
                                     x.question2),
               axis=1)
data["fuzz_WRatio"] = \
    data.apply(lambda x: fuzz.WRatio(x.question1,
                                     x.question2),
               axis=1)
# Calculate the partial ratio of each pair of questions
data["fuzz_partial_ratio"] = \
    data.apply(lambda x: fuzz.partial_ratio(x.question1,
                                            x.question2),
               axis=1)

In [10]:
# Calculate the partial token set ratio of each pair of questions
data["fuzz_partial_token_set_ratio"] = \
    data.apply(lambda x: fuzz.partial_token_set_ratio(x.question1,
                                                      x.question2),
               axis=1)
# Calculate the partial token sort ratio of each pair of questions
data["fuzz_partial_token_sort_ratio"] = \
    data.apply(lambda x: fuzz.partial_token_sort_ratio(x.question1,
                                                       x.question2),
               axis=1)

In [11]:
# Calculate the token set ratio of each pair of questions
data["fuzz_token_set_ratio"] = \
    data.apply(lambda x: fuzz.token_set_ratio(x.question1,
                                              x.question2),
               axis=1)
# Calculate the token sort ratio of each pair of questions
data["fuzz_token_sort_ratio"] = \
    data.apply(lambda x: fuzz.token_sort_ratio(x.question1,
                                               x.question2),
               axis=1)

In [12]:
# The distance-based feature set for future reference
fs_2 = ['fuzz_QRatio', 'fuzz_WRatio', 'fuzz_partial_ratio',
        'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
        'fuzz_token_set_ratio', 'fuzz_token_sort_ratio']

**TF-IDF & LSA BASED FEATURES**

In [13]:
# Create term frequency-inverse document frequency vectorizers
tfv = TfidfVectorizer(min_df=3,
                      max_features=None,
                      strip_accents='unicode',
                      analyzer='word',
                      token_pattern=r"\w{1,}",
                      ngram_range=(1, 2),
                      use_idf=1,
                      smooth_idf=1,
                      sublinear_tf=1,
                      stop_words="english")
tfv_q1 = deepcopy(tfv)
tfv_q2 = deepcopy(tfv)

In [14]:
# Calculate the tf-idf matrices for both questions
q1_tfidf = tfv_q1.fit_transform(data.question1.fillna(""))
q2_tfidf = tfv_q2.fit_transform(data.question2.fillna(""))

In [15]:
# Create truncated SVD decompostions = fast but aproximate, with 180 components
svd = TruncatedSVD(n_components=180)
svd_q1 = TruncatedSVD(n_components=180)
svd_q2 = TruncatedSVD(n_components=180)

In [16]:
# Calculate the SVD features based on the tf-idf matrices
question1_vectors = svd_q1.fit_transform(q1_tfidf)
question2_vectors = svd_q2.fit_transform(q2_tfidf)

In [17]:
# The 3rd feature set is obtained by combining the tf-idf and SVD features
# Stack the tf-idf matrices together
fs3_1 = sparse.hstack((q1_tfidf, q2_tfidf))

In [18]:
# First combine the questions and then calculate the tf-idf
q1q2 = data.question1.fillna("") \
     + " " \
     + data.question2.fillna("")
fs3_2 = tfv.fit_transform(q1q2)

In [19]:
# Stack the SVD matrices togetherr
fs3_3 = np.hstack((question1_vectors, question2_vectors))

In [20]:
# Calculate the skew and kurtosis of the question vectors
data['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

In [21]:
# Save the skew and kurtosis features for future reference
fs3_4 = ['skew_q1vec', 'skew_q2vec', 'kur_q1vec', 'kur_q2vec']

**WORD2VEC BASED FEATURES**

Simply put, Word2vec models are bi-layered neural networks that transform words to vectors so that words with similar meanings are close to eachother.
Word2vec models take a corpus as input and output a vector for each word in that corpus.

In [22]:
# Load Google's Word2vec model
model = gensim.models \
              .KeyedVectors \
              .load_word2vec_format("GoogleNews-vectors-negative300.bin.gz",
                                    binary=True)

In [23]:
# Google's Word2vec model expects words as input, so sentences must be
# transformed to vectors indirectly
def sent2vec(s, model):
    words = word_tokenize(s.lower())
    # Stopwords and numbers must be removed, as well as words that are not
    # part of the model
    M = [model[w] for w in words if w not in stop_words \
                             and w.isalpha() \
                             and w in model]
    M = np.array(M)
    if len(M) > 0:
        v = M.sum(axis=0)
        return v / np.sqrt((v ** 2).sum())
    else:
        # When the sentence is empty after removing unvalid tokens, the vector
        # is equal to the null-vector
        return model.get_vector('null')

In [24]:
# Calculate the sent2vec vectors for every question
w2v_q1 = np.array([sent2vec(q, model) 
                   for q in data.question1])
w2v_q2 = np.array([sent2vec(q, model) 
                   for q in data.question2])
# Stack the sent2vec vectors
w2v = np.hstack((w2v_q1, w2v_q2))

In [25]:
fs_5 = ['w2v']

In [26]:
# Calculate numerous sent2vec-based distance features
data["cosine_distance"] = [cosine(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]
data["cityblock_distance"] = [cityblock(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]
data["jaccard_distance"] = [jaccard(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]
data["canberra_distance"] = [canberra(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]
data["euclidean_distance"] = [euclidean(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]
data["minkowski_distance"] = [minkowski(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]
data["braycurtis_distance"] = [braycurtis(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]

In [27]:
# The sent2vec-distance-based feature set for future reference
fs4_1 = ['cosine_distance', 'cityblock_distance', 
         'jaccard_distance', 'canberra_distance', 
         'euclidean_distance', 'minkowski_distance',
         'braycurtis_distance']

In [28]:
# Finally calculate the word mover distance between two questions
def wmd(s1, s2, model):
    s1 = s1.lower().split()
    s2 = s2.lower().split()
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

In [29]:
# Calculate the wmd for each pair of questions
data["wmd"] = \
    data.apply(lambda x: wmd(x.question1, x.question2, model),
               axis=1)
model.init_sims(replace=True)
data["norm_wmd"] = \
    data.apply(lambda x: wmd(x.question1, x.question2, model),
               axis=1)

In [30]:
fs4_2 = ['wmd', 'norm_wmd']

# Build Machine Learning Model

In [31]:
# Normalize the data
scaler = StandardScaler()
y = data.is_duplicate.values
y = y.astype("float32").reshape(-1,1)
X = data[fs_1+fs_2+fs3_4+fs4_1+fs4_2]
X = X.replace([np.inf, -np.inf], np.nan).fillna(0).values
X = scaler.fit_transform(X)
X = np.hstack((X, fs3_3))

In [32]:
# Save the X and y arrays for later testing, so you don't have to rerun the entire file
# WARNING: the X.tsv file is almost 4GB in size
np.savetxt("y.tsv", y, delimiter="\t")
np.savetxt("X.tsv", X, delimiter="\t")

In [33]:
# Load the X and y arrays for testing, this takes quite some time due to the size of X.tsv
# y = np.genfromtxt("y.tsv", delimiter="\t")
# y = y.astype("float32").reshape(-1,1)
# X = np.genfromtxt("X.tsv", delimiter="\t")

In [34]:
# Split the data into training and test data
np.random.seed(42)
n_all, _ = y.shape
idx = np.arange(n_all)
np.random.shuffle(idx)
n_split = n_all // 10
idx_val = idx[:n_split]
idx_train = idx[n_split:]
x_train = X[idx_train]
y_train = np.ravel(y[idx_train])
x_val = X[idx_val]
y_val = np.ravel(y[idx_val])

In [35]:
# Build the xgboost model
params = dict()
params["objective"] = "binary:logistic"
params["eval_metric"] = ["logloss", "error"]
params["eta"] = 0.02
params["max_depth"] = 4
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_val, label=y_val)
watchlist = [(d_train, "train"), (d_valid, "valid")]

In [36]:
bst = xgb.train(params, d_train, 5000, watchlist,
                early_stopping_rounds=50, verbose_eval=100)
xgb_preds = (bst.predict(d_valid) >= 0.5).astype(int)
xgb_accuracy = np.sum(xgb_preds == y_val) / len(y_val)
print("Xgb accuracy : %0.3f" % xgb_accuracy)

[0]	train-logloss:0.687516	train-error:0.297336	valid-logloss:0.687544	valid-error:0.297583
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 50 rounds.
[100]	train-logloss:0.501831	train-error:0.260918	valid-logloss:0.503712	valid-error:0.262485
[200]	train-logloss:0.467492	train-error:0.243948	valid-logloss:0.470082	valid-error:0.246259
[300]	train-logloss:0.451474	train-error:0.234164	valid-logloss:0.454613	valid-error:0.235425
[400]	train-logloss:0.441249	train-error:0.227936	valid-logloss:0.445057	valid-error:0.231344
[500]	train-logloss:0.433691	train-error:0.222464	valid-logloss:0.438244	valid-error:0.22798
[600]	train-logloss:0.427585	train-error:0.218435	valid-logloss:0.432796	valid-error:0.224443
[700]	train-logloss:0.422411	train-error:0.215431	valid-logloss:0.428255	valid-error:0.220609
[800]	train-logloss:0.417999	train-error:0.212441	valid-logloss:0.424457	valid-error:0.217962
[900]	trai

In [37]:
# Save the model
bst.save_model("0001.model")

In [38]:
# Load the model
bst_loaded = xgb.Booster({"nthread":4})
bst_loaded.load_model("0001.model")

In [39]:
# Test whether the loaded model works
xgb_preds_test = (bst_loaded.predict(d_valid) >= 0.5).astype(int)
xgb_accuracy_test = np.sum(xgb_preds_test == y_val) / len(y_val)
print("Xgb accuracy : %0.3f" % xgb_accuracy_test)

Xgb accuracy : 0.804
